# Evaluasi Hasil Ekstraksi SkillNER

In [ ]:
# from sklearn.metrics import cohen_kappa_score

# rater1_ratings = [0, 1, 1, 0, 1, 0, 8, 0]
# rater2_ratings = [0, 1, 0, 0, 0, 1, 0, 4]

# rater1_ratings = ['cat', 'dog', 'cat', 'bird', 'dog']
# rater2_ratings = ['dog', 'dog', 'dog', 'bird', 'dog']

# kappa_score = cohen_kappa_score(rater1_ratings, rater2_ratings)
# print(f"Cohen's Kappa: {kappa_score}")

Cohen's Kappa: 0.1428571428571428


# Hitung Kappa 2 Pakar

In [4]:
import pandas as pd

pakar1_path = "docs/skillner evaluation/Anotasi Skill - Rizfi.xlsx"
pakar2_path = "docs/skillner evaluation/Anotasi Skill - Khansa.xlsx"
pakar3_path = "docs/skillner evaluation/Anotasi Skill - Naufal.xlsx"

sheets = "Job Posting"
name = "Nama Pekerjaan"

print(f"Kappa {sheets}:")
pakar1 = pd.read_excel(pakar1_path, sheet_name=sheets)
pakar2 = pd.read_excel(pakar2_path, sheet_name=sheets)
pakar3 = pd.read_excel(pakar3_path, sheet_name=sheets)

def normalize(skill):
    # return skill.strip().lower().replace('-', ' ').replace('_', ' ').replace('  ', ' ')
    return ' '.join(skill.strip().lower().replace('-', ' ').replace('_', ' ').replace('  ', ' ').split())

total_data = 20
total_score = 0
kappa_data = []
for i in range(total_data):
    a1 = {normalize(skill) for skill in pakar1.at[i, "Hasil Anotasi Pakar"].split("\n") if skill.strip()}
    a2 = {normalize(skill) for skill in pakar2.at[i, "Hasil Anotasi Pakar"].split("\n") if skill.strip()}
    a3 = {normalize(skill) for skill in pakar3.at[i, "Hasil Anotasi Pakar"].split("\n") if skill.strip()}

    # corpus = {normalize(skill) for skill in pakar1.at[i, "Korpus"].split("\n") if skill.strip()}
    corpus = {normalize(skill) for skill in pakar1.at[i, "Korpus"].split(" ") if skill.strip()}

    # Tentukan Kombinasi Pakar:
    # 1-2: Rizfi-Khansa [a1, a2]
    # 1-3: Rizfi-Naufal [a1, a3]
    # 2-3: Khansa-Naufal [a2, a3]
    anotators = [a1, a3]

    # Gabungan anotasi 2 pakar + corpus
    all_terms = sorted(set(anotators[0]).union(set(anotators[1])).union(set(corpus)))

    # all_terms = set()
    # for skills_list in union_anotators[0]["Hasil Anotasi Pakar"].astype(str).str.lower().str.split('\n'): all_terms.update(s for s in skills_list if s)
    # for skills_list in union_anotators[1]["Hasil Anotasi Pakar"].astype(str).str.lower().str.split('\n'): all_terms.update(s for s in skills_list if s)
    # all_terms = sorted(set(all_terms))
    N = len(all_terms)
    
    A, B, C, D = 0, 0, 0, 0
    for term in all_terms:
        if term in anotators[0] and term in anotators[1]:
            A += 1
            # print(f"A = {term}")
        elif term in anotators[0] and term not in anotators[1]:
            B += 1
            # print(f"B = {term}")
        elif term not in anotators[0] and term in anotators[1]:
            C += 1
            # print(f"C = {term}")
        elif term not in anotators[0] and term not in anotators[1]:
            D += 1
            # print(f"D = {term}")
    
    anotator1_yes = (A + B) / N
    anotator2_yes = (A + C) / N
    anotator1_no = (B + D) / N
    anotator2_no = (C + D) / N
    
    po = (A + D) / N
    pe = (anotator1_yes * anotator2_yes) + (anotator1_no * anotator2_no)
    
    kappa_score = (po - pe) / (1 - pe)
    
    # print(f"Nilai Kappa: {kappa_score:.4f} | Job {i+1}: {union_anotators[0].at[i, 'Nama Pekerjaan']}")

    total_score += kappa_score
    # print(f"Pekerjaan {i+1}")
    kappa_data.append({
        "nama": pakar1.at[i, name],
        "A": A,
        "B": B,
        "C": C,
        "D": D,
        "Po": po,
        "Pe": pe,
        "Nilai Kappa": kappa_score
    })
    # print(kappa_score)


print("=============")
print(f"Rata-rata Kappa Pasangan: {total_score/total_data:.4f}")
kappa_list = pd.DataFrame(kappa_data)
kappa_list

Kappa Job Posting:
Rata-rata Kappa Pasangan: 0.3906


,nama,A,B,C,D,Po,Pe,Nilai Kappa
0,Programmer,12,7,7,47,0.808219,0.614937,0.501949
1,IT Quality Assurance,10,18,14,103,0.779310,0.705303,0.251130
2,NET Developer,27,3,2,41,0.931507,0.518296,0.857811
3,Software Engineer,6,8,10,32,0.678571,0.607143,0.181818
4,Data Analyst Staff,10,10,11,61,0.771739,0.653592,0.341064
5,Backend Developer,9,13,6,61,0.786517,0.667592,0.357767
6,Data Engineer,14,8,6,49,0.818182,0.602968,0.542056
7,IT Auditor,9,14,12,76,0.765766,0.682006,0.263400
8,AI/ML Engineer,15,11,10,68,0.798077,0.629808,0.454545
9,Backend Developer (Golang),9,9,7,45,0.771429,0.631837,0.379157


In [30]:
import pandas as pd
import numpy as np

def normalize(skill):

    return skill.strip().lower().replace('-', ' ').replace('_', ' ').replace('  ', ' ')

def manual_cohen_kappa(rater1_vector, rater2_vector):

    # n_items = a + b + c + d
    n_items = len(rater1_vector)
    if n_items == 0:
        return 1.0

    # agreements = a + d
    agreements = sum(1 for i in range(n_items) if rater1_vector[i] == rater2_vector[i])
    po = agreements / n_items

    rater1_yes = sum(rater1_vector) / n_items
    rater1_no = 1 - rater1_yes
    
    rater2_yes = sum(rater2_vector) / n_items
    rater2_no = 1 - rater2_yes
    
    chance_yes = rater1_yes * rater2_yes
    chance_no = rater1_no * rater2_no
    
    pe = chance_yes + chance_no
    
    if pe == 1.0:
        return 1.0 if po == 1.0 else 0.0

    kappa = (po - pe) / (1 - pe)
    
    return kappa, rater1_yes, rater1_no, rater2_yes, rater2_no, po, pe

def calculate_and_print_kappa(file1, file2, sheet_name):

    print(f"--- Memproses Sheet: {sheet_name} ---")

    if sheet_name == 'Job Posting':
        id_column = 'Nama Pekerjaan'
    elif sheet_name == 'SFIA':
        id_column = 'Skill - Level'
    else:
        print(f"Nama sheet '{sheet_name}' tidak dikenali.")
        return

    try:
        df1 = pd.read_excel(file1, sheet_name=sheet_name)
        df2 = pd.read_excel(file2, sheet_name=sheet_name)
    except Exception as e:
        print(f"Gagal membaca sheet '{sheet_name}'. Pastikan nama sheet dan file sudah benar. Error: {e}")
        return

    on_column = 'Korpus'
    annotation_column = 'Hasil Anotasi Pakar'
    annotator1_name = file1.replace('Anotasi Skill - ', '').replace('.xlsx', '').lower()
    annotator2_name = file2.replace('Anotasi Skill - ', '').replace('.xlsx', '').lower()

    df1 = df1.rename(columns={annotation_column: f'{annotator1_name}_annotations'})
    df2 = df2.rename(columns={annotation_column: f'{annotator2_name}_annotations'})

    df1_subset = df1[[id_column, on_column, f'{annotator1_name}_annotations']].dropna(subset=[on_column])
    df2_subset = df2[[on_column, f'{annotator2_name}_annotations']].dropna(subset=[on_column])
    
    merged_df = pd.merge(df1_subset, df2_subset, on=on_column)

    results = []

    def get_normalized_annotated_skills(annotation):
        if pd.isna(annotation):
            return set()
        return {normalize(s) for s in str(annotation).split('\n')}

    for index, row in merged_df.iterrows():
        corpus_text = row[on_column]
        
        # Korpus dari deskripsi per baris
        items_to_rate = sorted(list(set(normalize(word) for word in str(corpus_text).split())))
        
        # Hasil anotasi per baris
        anot1_skills = get_normalized_annotated_skills(row[f'{annotator1_name}_annotations'])
        anot2_skills = get_normalized_annotated_skills(row[f'{annotator2_name}_annotations'])

        anot1_vector = [1 if item in anot1_skills else 0 for item in items_to_rate]
        anot2_vector = [1 if item in anot2_skills else 0 for item in items_to_rate]
        
        kappa, rater1_yes, rater1_no, rater2_yes, rater2_no, po, pe = manual_cohen_kappa(anot1_vector, anot2_vector)

        results.append({
            id_column: row[id_column],
            'kappa_score': kappa,
            'A': rater1_yes,
            'B': rater1_no,
            'C': rater2_yes,
            'D': rater2_no
        })

    results_df = pd.DataFrame(results)

    if results_df.empty:
        print("Tidak ada data yang cocok untuk diproses.")
        return
        
    avg_kappa = results_df['kappa_score'].mean()

    print(f"\nNilai Rata-rata Kappa: {avg_kappa:.4f}\n")
    print("Rincian Nilai Kappa per Baris (dengan normalisasi):")
    print(results_df.to_string())
    print("-" * 50 + "\n")


file_anot1 = "docs/skillner evaluation/Anotasi Skill - Naufal.xlsx"
file_anot2 = "docs/skillner evaluation/Anotasi Skill - Rizfi.xlsx"

calculate_and_print_kappa(file_anot1, file_anot2, sheet_name='Job Posting')

calculate_and_print_kappa(file_anot1, file_anot2, sheet_name='SFIA')

--- Memproses Sheet: Job Posting ---

Nilai Rata-rata Kappa: 0.5111

Rincian Nilai Kappa per Baris (dengan normalisasi):
                                                 Nama Pekerjaan  kappa_score         A         B         C         D
0                                                    Programmer     0.792727  0.157895  0.842105  0.140351  0.859649
1                                          IT Quality Assurance     0.171968  0.042017  0.957983  0.109244  0.890756
2                                                 NET Developer     0.873842  0.271186  0.728814  0.288136  0.711864
3                                             Software Engineer     0.265985  0.121951  0.878049  0.146341  0.853659
4                                            Data Analyst Staff     0.300567  0.094595  0.905405  0.121622  0.878378
5                                             Backend Developer     0.485714  0.069444  0.930556  0.138889  0.861111
6                                                 Data Engin

# Metrik Evaluasi (Belum Fix)

In [71]:
import pandas as pd

pakar1_path = "docs/skillner evaluation/Anotasi Skill - Rizfi.xlsx"
pakar2_path = "docs/skillner evaluation/Anotasi Skill - Khansa.xlsx"
pakar3_path = "docs/skillner evaluation/Anotasi Skill - Naufal.xlsx"

print("Evaluasi Job Posting:")
pakar1 = pd.read_excel(pakar1_path, sheet_name="SFIA")
pakar2 = pd.read_excel(pakar2_path, sheet_name="SFIA")
pakar3 = pd.read_excel(pakar3_path, sheet_name="SFIA")

total_precision = 0
total_recall = 0
total_f1score = 0
for i, skill in pakar1.iterrows():
    p1 = set(skill["Hasil Anotasi Pakar"].lower().split("\n"))
    p3 = set(pakar3.at[i, "Hasil Anotasi Pakar"].lower().split("\n"))

    # Data Ekstraksi SkillNER bisa memakai file mana saja, karena semuanya sama
    system = set(skill["Hasil Ekstraksi Sistem"].lower().split("\n"))

    # Bentuk Ground Truth Dari Kombinasi Pakar Sesuai Hasil Kappa Tertinggi (Pakar 1 & Pakar 3):
    union_gt = p1.union(p3)
    intersection_gt = p1.intersection(p3)

    # all_terms = set()
    # for skills_list in union_anotators[0]["Hasil Anotasi Pakar"].astype(str).str.lower().str.split('\n'): all_terms.update(s for s in skills_list if s)
    # for skills_list in union_anotators[1]["Hasil Anotasi Pakar"].astype(str).str.lower().str.split('\n'): all_terms.update(s for s in skills_list if s)
    # all_terms = sorted(set(all_terms))
    
    print("===== GABUNGAN =====")
    TP = len(system.intersection(union_gt))
    FP = len(system - union_gt)
    FN = len(union_gt - system)

    # print("===== IRISAN =====")
    # TP = len(system.intersection(intersection_gt))
    # FP = len(system - intersection_gt)
    # FN = len(intersection_gt - system)

    precision = TP / (TP + FP) if (TP+FP) > 0 else 0.0
    recall = TP / (TP + FN) if (TP+FN) > 0 else 0.0
    f1_score = (2 * precision * recall) / (precision + recall)  if (precision+recall) > 0 else 0.0

    total_precision += precision
    total_recall += recall
    total_f1score += f1_score

    print(f"Data SFIA {i+1} - {skill['Skill - Level']}:")
    print(f"TP = {TP}, FP = {FP}, FN = {FN}")
    print(f"Precision = {precision:.4f}, Recall = {recall:.4f}, F1-Score = {f1_score:.4f}")
    print("")

total_data = 20
print(f"Rata-rata Precision = {total_precision/total_data:.4f}, Rata-rata Recall = {total_recall/total_data:.4f}, Rata-rata F1-Score = {total_f1score/total_data:.4f}")

Evaluasi Job Posting:
===== GABUNGAN =====
Data SFIA 1 - Programming/software development - Level 4:
TP = 1, FP = 3, FN = 9
Precision = 0.2500, Recall = 0.1000, F1-Score = 0.1429

===== GABUNGAN =====
Data SFIA 2 - Data science - Level 6:
TP = 1, FP = 6, FN = 8
Precision = 0.1429, Recall = 0.1111, F1-Score = 0.1250

===== GABUNGAN =====
Data SFIA 3 - Machine learning - Level 4:
TP = 1, FP = 9, FN = 13
Precision = 0.1000, Recall = 0.0714, F1-Score = 0.0833

===== GABUNGAN =====
Data SFIA 4 - Data analytics - Level 7:
TP = 1, FP = 9, FN = 12
Precision = 0.1000, Recall = 0.0769, F1-Score = 0.0870

===== GABUNGAN =====
Data SFIA 5 - Data engineering - Level 4:
TP = 1, FP = 9, FN = 15
Precision = 0.1000, Recall = 0.0625, F1-Score = 0.0769

===== GABUNGAN =====
Data SFIA 6 - Data visualisation - Level 5:
TP = 1, FP = 9, FN = 10
Precision = 0.1000, Recall = 0.0909, F1-Score = 0.0952

===== GABUNGAN =====
Data SFIA 7 - Graphic design - Level 4:
TP = 1, FP = 12, FN = 17
Precision = 0.0769, Reca

# Corpus Generator

In [7]:
# Generate Corpus
import pandas as pd, re, nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    
    if not isinstance(text, str):
        return ''
    # Ubah teks menjadi huruf kecil
    text = text.lower()
    # Hapus tanda baca
    text = re.sub(r'[().,\-\']', ' ', text)
    # Hapus angka
    text = re.sub(r'\d+', '', text)
    # Tokenisasi teks
    tokens = word_tokenize(text)
    # Hapus stopwords
    tokens = [word for word in tokens if word not in stop_words]
    return '\n'.join(tokens)

file_path = "docs/skillner evaluation/Anotasi Skill - Naufal.xlsx"
job_posting = pd.read_excel(file_path, sheet_name="Job Posting")
sfia = pd.read_excel(file_path, sheet_name="SFIA")

job_posting["Korpus"] = job_posting["Deskripsi Pekerjaan"].apply(preprocess_text)
sfia["Korpus"] = sfia["Deskripsi Level"].apply(preprocess_text)

with pd.ExcelWriter(file_path) as writer:
    job_posting.to_excel(writer, sheet_name="Job Posting", index=False)
    sfia.to_excel(writer, sheet_name="SFIA", index=False)